<a href="https://colab.research.google.com/github/Nawazwariya182/Nawazwariya182/blob/main/Pokemon_Image_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# import zipfile
# import os

# zip_path = '/content/drive/MyDrive/ds.zip'  # update this
# extract_to = '/content'

# with zipfile.ZipFile(zip_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_to)

# print("✅ ZIP extracted to /content")


In [ ]:
# pyright: reportMissingImports=false
!pip install gradio
!pip install tensorflow
!pip install pillow
import google.colab
IN_COLAB = 'google.colab' in str(google.colab.__file__)
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.optimizers import Adam
from PIL import Image
import gradio as gr
import time

In [ ]:
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

try:
    policy = tf.keras.mixed_precision.Policy('mixed_float16')
    tf.keras.mixed_precision.set_global_policy(policy)
    print('Mixed precision enabled')
except:
    print('Mixed precision not enabled')

In [ ]:
def create_model(num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
    base_model.trainable = False

    model = Sequential([
        base_model,
        GlobalAveragePooling2D(),
        Dense(512, activation='relu'),
        Dropout(0.5),
        Dense(num_classes, activation='softmax')
    ])

    model.compile(
        optimizer=Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )

    return model

In [ ]:
def load_data(data_dir):
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(128, 128),
        batch_size=128
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(128, 128),
        batch_size=128
    )

    class_names = train_ds.class_names

    normalization_layer = tf.keras.layers.Rescaling(1./255)

    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    return train_ds, val_ds, class_names


In [ ]:
def train_model(data_dir, model_save_path='pokemon_model.h5'):
    print(f"Loading data from: {data_dir}")
    train_ds, val_ds, class_names = load_data(data_dir)
    num_classes = len(class_names)
    print(f"Training with {num_classes} Pokémon classes")

    model = create_model(num_classes)

    # Callbacks configuration
    callbacks = [
        # Early stopping after no improvement for 10 epochs
        tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True),

        # Reduce learning rate when validation loss plateaus
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001),

        # Save the best model based on validation accuracy
        tf.keras.callbacks.ModelCheckpoint(model_save_path, save_best_only=True, monitor='val_accuracy')
    ]

    start_time = time.time()

    model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=50,  # Adjusted epochs to 100
        callbacks=callbacks
    )

    training_time = time.time() - start_time
    print(f"Training completed in {training_time/60:.2f} minutes")

    model.save(model_save_path)

    return model, class_names


In [ ]:

def predict_pokemon(img, model_path='pokemon_model.h5', class_names=None):
    if isinstance(img, np.ndarray):
        img_pil = Image.fromarray(img.astype('uint8'))
    else:
        img_pil = Image.open(img).convert('RGB')

    img_pil = img_pil.resize((128, 128))
    img_array = np.array(img_pil) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    model = load_model(model_path)
    prediction = model.predict(img_array)
    predicted_class_idx = np.argmax(prediction[0])

    if class_names:
        return class_names[predicted_class_idx]
    else:
        return f"Class index: {predicted_class_idx}"

In [ ]:
def setup_gradio_interface(model_path='pokemon_model.h5', class_names=None):
    def predict_image(image):
        return predict_pokemon(image, model_path, class_names)

    interface = gr.Interface(
        fn=predict_image,
        inputs=gr.Image(),
        outputs=gr.Textbox(label="Predicted Pokémon"),
        title="Pokémon Classifier",
        description="Upload an image of a Pokémon and get a prediction"
    )

    return interface

In [ ]:
if __name__ == "__main__":
    data_dir = "/content/ds/pokemon-dataset-1000/dataset/"
    model_path = "pokemon_model.h5"

    if not os.path.exists(model_path):
        try:
            folders = os.listdir(data_dir)
            print(f"Found {len(folders)} Pokémon classes in the dataset")
            if len(folders) > 0:
                print(f"Sample classes: {folders[:5]}")
        except Exception as e:
            print(f"Error accessing dataset: {e}")

        model, class_names = train_model(data_dir, model_path)
    else:
        _, _, class_names = load_data(data_dir)

    interface = setup_gradio_interface(model_path, class_names)
    interface.launch(share=True if IN_COLAB else False)

In [ ]:
import json
import tensorflow as tf

def load_data1(data_dir):
    # Load training and validation datasets
    train_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=123,
        image_size=(128, 128),
        batch_size=128
    )

    val_ds = tf.keras.utils.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=123,
        image_size=(128, 128),
        batch_size=128
    )

    # Get the class names
    class_names = train_ds.class_names

    # Save the class names to a JSON file
    with open('class_names.json', 'w') as f:
        json.dump(class_names, f)

    # Normalize the images
    normalization_layer = tf.keras.layers.Rescaling(1./255)

    # Apply the normalization to the datasets
    train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
    val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

    # Cache and prefetch for better performance
    AUTOTUNE = tf.data.AUTOTUNE
    train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

    return train_ds, val_ds, class_names
